In [1]:
import pandas as pd
import seaborn as sns
import glob
from numpy import genfromtxt
# from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import numpy as np
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
import json
from collections import OrderedDict
import os
import re
import logging
import multiprocessing
from functools import partial
from datetime import datetime
from Bio import SeqIO
from Bio.Seq import Seq
import gzip
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold

/data/hoan/mybin/miniconda3/envs/py36/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Create map from gene ID to cluster ID

In [2]:
# panta input directory
# pantain_dir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokkatest/'
pantain_dir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokka/'
# panta output dir
# pantaout_dir = '/data/hoan/amromics/prediction/output/pantaEcoli1936aligntest/'
pantaout_dir = '/data/hoan/amromics/prediction/output/pantaEcoli1936align_v4/'

In [3]:
with open(pantaout_dir + 'annotated_clusters.json', 'r') as JSON:
    json_dict = json.load(JSON)
# data = json.loads('/data/hoan/amromics/prediction/output/pantaEcoli1936aligntest/clusters.json')[0]

In [4]:
# json_dict

In [5]:
gene2clusterdict = {}
for key in json_dict:
    if len(json_dict[key])==0:
        gene2clusterdict[key] = key
    for gene in json_dict[key]['gene_id']:
        gene2clusterdict[gene] = key

### Find all AMR genes

In [6]:
def parse_gff_AMRgene_finder(gff_fh, sample_id, min_protein_len=40):
    # gene_annotation = OrderedDict()
    # gene_position = OrderedDict()    
    # suffix = 1
    # bed_records = []
    # gene_index = 0
    seq_id = None
    min_cds_len = 3 * min_protein_len
    gene_list = []
    
    for line in gff_fh:            
        if line.startswith('##FASTA'):
            #Done reading gff, move on to reading fasta
            break

        if line[0] == '#':
            continue
        line = line.strip()
        #print(line)
        cells = line.split('\t')
        if cells[2] != 'CDS':
            continue
        if 'BARRGD' not in cells[8]:
            continue
        start = int(cells[3])
        end = int(cells[4])
        length = end - start + 1
        if length < min_cds_len:
            continue
        if length % 3 != 0:
            continue
        cells[0] = cells[0].replace('-','_') #make sure seq_id has no -
        
        if seq_id != cells[0]:
            seq_id = cells[0]
            gene_index = 0

        # strand = cells[6]
        tags = cells[8].split(';')
        gene_id = None
        gene_name = ''
        gene_product = ''
        for tag in tags:
            if tag.startswith('ID='):
                gene_id = tag[3:]
            elif tag.startswith('gene='):                    
                gene_name = tag[5:]
                gene_name = re.sub(r'\W', '_', gene_name)
            elif tag.startswith('product='):                    
                gene_product = tag[8:]
        if gene_id == None:
            continue

        # Ensure gene_id is in the format of sample_id-seq_id-gene_tag
        if not gene_id.startswith(sample_id + '-'):
            gene_id = sample_id + '-' + gene_id

        if not gene_id.startswith(sample_id + '-' + seq_id + '-'):
            gene_id = sample_id + '-' + seq_id + '-' + gene_id[len(sample_id)+1:]

        gene_list.append(gene_id)
    
    return gene_list

In [7]:
amr_gene = []
for data_dir in glob.glob(pantain_dir + '*.gff'):
    # print(data_dir)
    in_fh = open(data_dir)
    sample_id = data_dir.split('/')[-1][:-4]
    amr_gene += parse_gff_AMRgene_finder(in_fh, sample_id)
    in_fh.close()

In [8]:
amr_gene[:3], len(amr_gene)

(['SAMEA2204230.contig-SAMEA2204230.contig00001-KJJADFBE_00063',
  'SAMEA2204230.contig-SAMEA2204230.contig00001-KJJADFBE_00095',
  'SAMEA2204230.contig-SAMEA2204230.contig00001-KJJADFBE_00151'],
 119509)

## TODO: Map genes back to cluster IDs

In [9]:
amr_clusterID = [gene2clusterdict[gene] for gene in amr_gene]
amr_clusterID = list(set(amr_clusterID))

In [10]:
amr_clusterID[:3]

['vanR_I_1_15559', 'erm_X_', 'tcmA_12115']

In [11]:
len(amr_clusterID)

464

### Common cluster from matrix

In [45]:
pa_matrix = pd.read_csv(pantaout_dir+'gene_presence_absence.Rtab', sep='\t', index_col=0).T
# sel = VarianceThreshold(threshold=0)
# pa_matrix = sel.fit_transform(pa_matrix)

In [40]:
# pa_matrix.head(2)

In [41]:
n_samples = pa_matrix.shape[0]
n_genes = pa_matrix.shape[1]
n_genes

73473

In [42]:
colsum = pa_matrix.sum()

In [ ]:
common_gene_cluster = [colsum.index[idx] for idx in range(n_genes) if colsum[idx] > 0.99*n_samples]

In [44]:
common_gene_cluster[:4], len(common_gene_cluster)

(['rsmI', 'ssb_1', 'rpnA', 'rluF'], 2114)

### Top correlation features

In [76]:
pa_matrix = pd.read_csv(pantaout_dir+'gene_presence_absence.Rtab', sep='\t', index_col=0).T
colums_feature = pa_matrix.columns
sel = VarianceThreshold(threshold=0)
pa_matrix = sel.fit_transform(pa_matrix)
selected_columns = colums_feature[sel.get_support(indices=True)]

In [46]:
metadata_panta = pd.read_csv("data/Ecoli1936metafiles/metadata_final.csv")

In [47]:
metadata_panta.head(2)

,Isolate,Year,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,TBM,TMP,CIP
0,11658_4#1,2006.0,S,S,S,NaN,S,S,R,S,S,S,S,S
1,11657_5#1,2006.0,S,S,R,NaN,R,S,S,S,S,S,R,R


In [52]:
from pangraph.utils import binary_label
from sklearn.feature_selection import mutual_info_classif, chi2

In [88]:
mutual_mat = []
for idx in range(2, 14):
    y_class = metadata_panta.iloc[:,idx].values
    # print(metadata_panta.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = pa_matrix[nonenan_index, ]
    y_new = y[nonenan_index].astype(int)
    scores, pvalue = chi2(pa_matrix_new, y_new)
    mutual_mat.append(scores)
mutual_mat = np.array(mutual_mat)

In [89]:
mutual_mat_max = mutual_mat.max(axis=0)

In [90]:
df_feature = pd.DataFrame({'gene': selected_columns, 'score': mutual_mat_max})
df_feature = df_feature.sort_values(by='score', ascending=False)

In [91]:
topfeaturesbypresenceabsence = df_feature['gene'][:400].values

### Top correlation features from Sim Matrix

In [110]:
pa_matrix = pd.read_csv(pantaout_dir+'gene_presence_absence.Rtab', sep='\t', index_col=0).T
n_samples = pa_matrix.shape[0]
n_genes = pa_matrix.shape[1]
colsum = pa_matrix.sum()
common_gene_cluster = [colsum.index[idx] for idx in range(n_genes) if colsum[idx] > 0.1*n_samples]

In [108]:
simmatrix = np.load(pantaout_dir + 'similarsite.npy')

In [111]:
simmatrix.shape, len(common_gene_cluster)

((1653, 7639), 7639)

In [27]:
metadata_panta = pd.read_csv("data/Ecoli1936metafiles/metadata_final.csv")

In [113]:
metadata_panta.head(2)

,Isolate,Year,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,TBM,TMP,CIP
0,11658_4#1,2006.0,S,S,S,NaN,S,S,R,S,S,S,S,S
1,11657_5#1,2006.0,S,S,R,NaN,R,S,S,S,S,S,R,R


In [29]:
from pangraph.utils import binary_label
from sklearn.feature_selection import mutual_info_classif, chi2, f_classif

In [118]:
mutual_mat = []
for idx in range(2, 14):
    y_class = metadata_panta.iloc[:,idx].values
    # print(metadata_panta.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = simmatrix[nonenan_index, ]
    y_new = y[nonenan_index].astype(int)
    scores, pvalue = chi2(pa_matrix_new, y_new)
    mutual_mat.append(scores)
mutual_mat = np.array(mutual_mat)

In [119]:
mutual_mat_max = mutual_mat.max(axis=0)

In [120]:
df_feature = pd.DataFrame({'gene': common_gene_cluster, 'score': mutual_mat_max})
df_feature = df_feature.sort_values(by='score', ascending=False)

In [124]:
# df_feature

In [126]:
topfeaturesbysimmat = df_feature['gene'][:400].values

### Compute label encoder for gene cluster

In [12]:
# Read prepresentative sequence
from Bio import SeqIO
genecluster2representativeseq = {}
with open(pantaout_dir+'representative_clusters_prot.fasta') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        name, sequence = record.id, str(record.seq)
        genecluster2representativeseq[name] = sequence
        # print(name,'----', sequence)

In [13]:
with open(pantaout_dir + 'samples.json', 'r') as JSON:
    sample_dict = json.load(JSON)
sample2integerindex = {}
for idx in range(len(sample_dict)):
    sample2integerindex[sample_dict[idx]['id']] = idx
n_samples = len(sample_dict)

In [14]:
# computed_gene_cluster = common_gene_cluster; #amr_clusterID
computed_gene_cluster = amr_clusterID;
# computed_gene_cluster = topfeaturesbypresenceabsence;
# computed_gene_cluster = topfeaturesbysimmat;

In [15]:
# amr_mat = None;
ksize = 20; # k = 10 for protein, 20 for DNA
kmer_list = [];
amr_mat = np.zeros((n_samples, len(computed_gene_cluster)))
pairdata = []
for idx in range(len(computed_gene_cluster)):
    # alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.fna.aln.gz'
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            seqraw = sequence.replace('-', '')
            n_kmers = len(seqraw) - ksize + 1
            for i in range(n_kmers):
                kmer = seqraw[i:i + ksize]
                kmer_list.append(kmer)
                pairdata.append((sample2integerindex[sample_id], kmer))

In [16]:
unique_kmer = list(set(kmer_list))

In [17]:
# AMR genes: (74198447, 169115)
len(kmer_list), len(unique_kmer)

(224127453, 765533)

In [18]:
kmer2index = {}
for i in range(len(unique_kmer)):
    kmer2index[unique_kmer[i]] = i

In [19]:
kmer_matrix = np.zeros((n_samples, len(unique_kmer)))

In [20]:
# ct = 0
for u, v in pairdata:
    # kmer_matrix[u, kmer2index[v]] += 1
    kmer_matrix[u, kmer2index[v]] = 1

In [21]:
# np.save(pantaout_dir + 'kmer_amr_mat.npy', kmer_matrix) # save numpy array

In [22]:
kmer_matrix.shape

(1653, 765533)

In [23]:
selector = VarianceThreshold(threshold=0.05)
kmer_matrix_VT = selector.fit_transform(kmer_matrix)

In [24]:
kmer_matrix_VT.shape

In [31]:
mutual_mat = []
for idx in range(2, 14):
    y_class = metadata_panta.iloc[:,idx].values
    print(metadata_panta.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = kmer_matrix_VT[nonenan_index, ]
    y_new = y[nonenan_index].astype(int)
    scores, pvalue = chi2(pa_matrix_new, y_new)
    mutual_mat.append(scores)
mutual_mat = np.array(mutual_mat)

In [32]:
mutual_mat_mean = mutual_mat.mean(axis=0)

In [34]:
top_features = np.argsort(mutual_mat_mean)[::-1][:100000]
kmer_matrix_VT_top_features = kmer_matrix_VT[:,top_features]
kmer_matrix_VT_top_features.shape

(1653, 100000)

In [35]:
# np.save(pantaout_dir + 'kmer_amr_mat_VT1.npy', kmer_matrix_VT) # save numpy array
# np.save(pantaout_dir + 'kmer_coregene_mat_VT1.npy', kmer_matrix_VT) # save numpy array
# np.save(pantaout_dir + 'topfeaturesbypresenceabsence_VT5.npy', kmer_matrix_VT) # save numpy array
# np.save(pantaout_dir + 'topfeaturesbysimmat_VT5.npy', kmer_matrix_VT) # save numpy array
# np.save(pantaout_dir + 'kmer_amr_count_mat_VT1.npy', kmer_matrix_VT) # save numpy array
# np.save(pantaout_dir + 'kmer_amr_DNA_mat_VT5.npy', kmer_matrix_VT) # save numpy array
np.save(pantaout_dir + 'kmer_amr_DNA_mat_VT5_top_features.npy', kmer_matrix_VT_top_features) # save numpy array

In [26]:
stop

NameError: name 'stop' is not defined

In [ ]:
def count_differ_char(seq1, seq2):
    count = 0
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            count += 1
    return(count)

In [ ]:
def count_similar_char(seq1, seq2):
    count = 0
    for i in range(len(seq1)):
        if seq1[i] == seq2[i]:
            count += 1
    return(count)

In [ ]:
# count_differ_char('DovanHoan', 'DoVANHOAN')

In [ ]:
# amr_mat = None;
amr_mat = np.zeros((n_samples, len(computed_gene_cluster)))
for idx in range(len(computed_gene_cluster)):
    alignment_dir = pantaout_dir + 'clusters/' + computed_gene_cluster[idx] +'/'+computed_gene_cluster[idx]+'.faa.aln.gz'
    # mat = None
    # index = 0
    # index_set = []
    # print(idx, end = ',')
    with gzip.open(alignment_dir, "rt") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            name, sequence = record.id, str(record.seq)
            sample_id = name.split('-')[0]
            amr_mat[sample2integerindex[sample_id], idx] = count_similar_char(genecluster2representativeseq[computed_gene_cluster[idx]], sequence)
            # index_set.append(sample2integerindex[sample_id])
            # print(record.id)
            # print(genecluster2representativeseq[computed_gene_cluster[idx]],':::::', sequence)

In [ ]:
# amrgene_annotation = pd.DataFrame({'gene': amr_clusterID, 'start_index': start_idx, 'end_index': end_idx})

In [ ]:
# amrgene_annotation.head(2)

In [ ]:
# amrgene_annotation.to_csv(pantaout_dir + 'amrgene_annotation_VarianceThreshold.csv', index=None)
# amrgene_annotation.to_csv(pantaout_dir + 'amrgene_annotation.csv', index=None)

In [ ]:
# pd.DataFrame(amr_mat).to_csv(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.csv', index=None)

In [ ]:
# amrtest = pd.read_csv(pantaout_dir + 'amrlabelencodermat.csv')
# np.save(pantaout_dir + 'amrlabelencodermat.npy', amrtest.values) # save numpy array

In [ ]:
# amrtest.values[:3,:3]

In [ ]:
# Full matrix shape: (1653, 174005)
# amr_mat.shape

In [ ]:
# np.save(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.npy', amr_mat) # save numpy array
np.save(pantaout_dir + 'similarsitecolsum1pct.npy', amr_mat) # save numpy array
# np.save(pantaout_dir + 'amrlabelencodermat.npy', amr_mat) # save numpy array

In [ ]:
stop

In [ ]:
# load data
amr_mat = np.load(pantaout_dir + 'amrlabelencodermat_VT10.npy')

In [ ]:
amr_mat.shape, np.max(amr_mat[:50,:3000])

In [ ]:
def one_hot_encode(seq):
    mapping = dict(zip("ACGT", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]

one_hot_encode("AACGT")

In [ ]:
mapping = dict()
for i in range(22):
    mapping[i] = i
def one_hot_encode2(seq):
    seq2 = [mapping[i] for i in seq]
    return np.eye(22)[seq2].flatten()

# one_hot_encode([1,2,3])

In [ ]:
amr_mat = amr_mat.astype(int)
amr_matOnehot = None
for idx in range(amr_mat.shape[0]):
    if idx == 0:
        amr_matOnehot = one_hot_encode2(amr_mat[idx,:])
    else:
        amr_matOnehot = np.vstack([amr_matOnehot, one_hot_encode2(amr_mat[idx,:])])

In [ ]:
amr_matOnehot.shape, amr_mat.shape

In [ ]:
amr_matOnehot